In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.34.0"

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Repositories count: 5
name: central url: https://repo.maven.apache.org/maven2/ release:true update:never snapshot:false update:never 
name: jcenter url: https://jcenter.bintray.com/ release:true update:never snapshot:false update:never 
name: jboss url: https://repository.jboss.org/nexus/content/repositories/releases/ release:true update:never snapshot:false update:never 
name: atlassian url: https://packages.atlassian.com/maven/public release:true update:never snapshot:false update:never 
name: local url: file:///Users/bsorrentino/.m2/repository/ release:true update:never snapshot:true update:always 


In [3]:
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}
%dependency /list-dependencies
%dependency /resolve

Adding dependency org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.34.0
Adding dependency dev.langchain4j:langchain4j-open-ai:0.34.0
Proposed dependencies count: 3
 - org.bsc.langgraph4j:langgraph4j-core-jdk8:jar:1.0-SNAPSHOT (runtime)
 - dev.langchain4j:langchain4j:jar:0.34.0 (runtime)
 - dev.langchain4j:langchain4j-open-ai:jar:0.34.0 (runtime)
Resolved dependencies count: 0
Solving dependencies
Resolved artifacts count: 25
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core-jdk8/1.0-SNAPSHOT/langgraph4j-core-jdk8-1.0-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/async/async-generator-jdk8/2.0.1/async-generator-jdk8-2.0.1.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /User

In [12]:
import org.bsc.langgraph4j.*;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.state.Channel;
import org.bsc.langgraph4j.state.AppenderChannel;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.data.message.ChatMessage;
import org.bsc.langgraph4j.action.AsyncNodeAction;
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;
import static org.bsc.langgraph4j.utils.CollectionsUtils.mapOf;
import org.bsc.langgraph4j.checkpoint.MemorySaver;
import org.bsc.langgraph4j.CompileConfig;
import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;

public class State extends AgentState {

    static Map<String, Channel<?>> SCHEMA = Map.of(
            "messages", AppenderChannel.<AiMessage>of(ArrayList::new)
    );

    public State(Map<String, Object> initData) {
        super( initData  );
    }

    Optional<String> input() { return value("input"); } 
    Optional<String> userFeedback() { return value("user_feedback"); } 

}

AsyncNodeAction<State> step1 = node_async(state -> {
    System.out.println( "---Step 1---" );
    return mapOf();
});

AsyncNodeAction<State> humanFeedback = node_async(state -> {
    System.out.println( "---human_feedback---" );
    return mapOf();
});

AsyncNodeAction<State> step3 = node_async(state -> {
    System.out.println( "---Step 3---" );
    return mapOf();
});

var builder = new StateGraph<>(State.SCHEMA, State::new);
builder.addNode("step_1", step1);
builder.addNode("human_feedback", humanFeedback);
builder.addNode("step_3", step3);
builder.addEdge(START, "step_1");
builder.addEdge("step_1", "human_feedback");
builder.addEdge("human_feedback", "step_3");
builder.addEdge("step_3", END);

// Set up memory
var saver = new MemorySaver();

// Add
var compileConfig = CompileConfig.builder().checkpointSaver(saver).interruptBefore("human_feedback").build();
var graph = builder.compile(compileConfig);

// View as Mermaid with ```mermaid ... ```
graph.getGraph(GraphRepresentation.Type.MERMAID).getContent();

---
title: Graph Diagram
---
flowchart TD
	start((start))
	stop((stop))
	step_1("step_1")
	human_feedback("human_feedback")
	step_3("step_3")
	start:::start --> step_1:::step_1
	step_1:::step_1 --> human_feedback:::human_feedback
	human_feedback:::human_feedback --> step_3:::step_3
	step_3:::step_3 --> __END__:::__END__


In [13]:
// Input
var initialInput = mapOf("input", (Object) "hello world");

// Thread
var invokeConfig = RunnableConfig.builder().threadId("Thread1").build();

// Run the graph until the first interruption
for (var event : graph.stream(initialInput, invokeConfig)) {
    System.out.println(event);
}


---Step 1---
NodeOutput(node=__START__, state={input=hello world, messages=[]})
NodeOutput(node=step_1, state={input=hello world, messages=[]})


In [14]:
// Get user input
//String userInput = new Scanner(System.in).nextLine();
String userInput = "go to step 3!";
System.out.println("Tell me how you want to update the state: " + userInput);

// We now update the state as if we are the human_feedback node
//var updateConfig = graph.updateState(invokeConfig, mapOf("user_feedback", userInput), "human_feedback");
var updateConfig = graph.updateState(invokeConfig, mapOf("user_feedback", userInput), null);

// We can check the state
System.out.println("--State after update--");
System.out.println(graph.getState(invokeConfig));

// We can check the next node, showing that it is node 3 (which follows human_feedback)
System.out.println("getNext with invokeConfig: " + graph.getState(invokeConfig).getNext());
System.out.println("getNext with updateConfig: " + graph.getState(updateConfig).getNext());

Tell me how you want to update the state: go to step 3!
--State after update--
StateSnapshot(state={user_feedback=go to step 3!, input=hello world, messages=[]}, config=RunnableConfig(threadId=Thread1, checkPointId=9cc155ba-c7a2-4b8a-81a0-fda9240e4bdd, nextNode=human_feedback))
getNext with invokeConfig: human_feedback
getNext with updateConfig: human_feedback


In [15]:
// Continue the graph execution
for (var event : graph.stream(null, updateConfig)) {
    System.out.println(event);
}

---human_feedback---
---Step 3---
NodeOutput(node=human_feedback, state={user_feedback=go to step 3!, input=hello world, messages=[]})
NodeOutput(node=step_3, state={user_feedback=go to step 3!, input=hello world, messages=[]})
NodeOutput(node=__END__, state={user_feedback=go to step 3!, input=hello world, messages=[]})


In [16]:
graph.getState(updateConfig).getState();

{user_feedback=go to step 3!, input=hello world, messages=[]}